In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: False


# Data Preprocessing

In [3]:
import os
os.chdir('..')

df = pd.read_csv('./hourly02-ithaca/hourly02-NY_Ithaca_13_E.csv', header = 0, index_col = 0)
Date = pd.to_datetime(df.UTC_DATE, format='%Y%m%d', errors='coerce')
+ pd.to_timedelta(df.UTC_TIME//100, unit = 'hours')
df['Time'] = Date
data = df[['T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_MAX',
       'SOLARAD_MIN', 'SUR_TEMP',
           'SUR_TEMP_MAX', 'SUR_TEMP_MIN', 'RH_HR_AVG']]
data.index = df['Time']

In [4]:
data.isna().sum()

T_CALC           1384
T_HR_AVG         1446
T_MAX            1385
T_MIN            1389
P_CALC            832
SOLARAD           570
SOLARAD_MAX      9757
SOLARAD_MIN      9757
SUR_TEMP          724
SUR_TEMP_MAX     9911
SUR_TEMP_MIN     9911
RH_HR_AVG       48248
dtype: int64

In [5]:
data.shape

(166759, 12)

In [6]:
# forward fill the missing values  
data.ffill(axis = 0, inplace = True)
# drop NaN at the top
data.dropna(inplace = True)

In [7]:
# set target
data['target'] = data['T_HR_AVG']

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [9]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

with open('./LSTM/models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [10]:
train.shape

(126057, 13)

In [11]:
test.shape

(31515, 13)

In [12]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, hidden_unit = 30, activation_type = 'tanh', name = 'BiLSTM'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = Bidirectional(LSTM(units=hidden_unit, activation = activation_type, input_shape = input_shape, return_sequences=False, name = 'lstm_1'), name = 'bilstm_1')
        self.dense1 = Dense(units=output_shape, activation = 'sigmoid', name = 'dense_1')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x
    
    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [14]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [15]:
X_train.shape

TensorShape([126037, 16, 12])

In [16]:
y_train.shape

TensorShape([126037, 4])

In [17]:
# create model instance
model_name = 'BiLSTM_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_16-4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 4)                 244       
                                                                 
Total params: 10,564
Trainable params: 10,564
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 9s 6ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5727 - val_loss: 5.4724e-04 - val_mse: 5.4724e-04 - val_acc: 0.5736
Epoch 2/30
1135/1135 [==============================] - 7s 6ms/step - loss: 5.8210e-04 - mse: 5.8210e-04 - acc: 0.6293 - val_loss: 4.9174e-04 - val_mse: 4.9174e-04 - val_acc: 0.6262
Epoch 3/30
1135/1135 [==============================] - 6s 6ms/step - loss: 5.3807e-04 - mse: 5.3807e-04 - acc: 0.6426 - val_loss: 4.9200e-04 - val_mse: 4.9200e-04 - val_acc: 0.5632
Epoch 4/30
1135/1135 [==============================] - 7s 6ms/step - loss: 5.1388e-04 - mse: 5.1388e-04 - acc: 0.6487 - val_loss: 4.6915e-04 - val_mse: 4.6915e-04 - val_acc: 0.6063
Epoch 5/30
1135/1135 [==============================] - 7s 6ms/step - loss: 4.9851e-04 - mse: 4.9851e-04 - acc: 0.6524 - val_loss: 4.7251e-04 - val_mse: 4.7251e-04 - val_acc: 0.6420
Epoch 6/30
1135/1135 [==============================] - 7s 6ms/step - loss: 4.8966e-04 - mse: 4.89

In [19]:
# save trained model
model.save('./LSTM/models/' + model_name)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_16-4\assets


985/985 [==============================] - 1s 1ms/step
mean_squared_error
train set: 0.000393290856248061
test set: 0.0004359774272130303


## input length : output length = 24:6

In [20]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [21]:
# create model instance
model_name = 'BiLSTM_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_24-6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 6)                 366       
                                                                 
Total params: 10,686
Trainable params: 10,686
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 14s 10ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.5455 - val_loss: 8.8443e-04 - val_mse: 8.8443e-04 - val_acc: 0.5704
Epoch 2/30
1135/1135 [==============================] - 9s 8ms/step - loss: 8.3685e-04 - mse: 8.3685e-04 - acc: 0.5890 - val_loss: 9.3538e-04 - val_mse: 9.3538e-04 - val_acc: 0.4488
Epoch 3/30
1135/1135 [==============================] - 10s 8ms/step - loss: 7.7153e-04 - mse: 7.7153e-04 - acc: 0.6030 - val_loss: 9.5088e-04 - val_mse: 9.5088e-04 - val_acc: 0.5700
Epoch 4/30
1135/1135 [==============================] - 10s 8ms/step - loss: 7.3620e-04 - mse: 7.3620e-04 - acc: 0.6100 - val_loss: 9.2715e-04 - val_mse: 9.2715e-04 - val_acc: 0.6086
Epoch 5/30
1135/1135 [==============================] - 10s 8ms/step - loss: 7.1377e-04 - mse: 7.1377e-04 - acc: 0.6138 - val_loss: 7.0248e-04 - val_mse: 7.0248e-04 - val_acc: 0.5055
Epoch 6/30
1135/1135 [==============================] - 10s 8ms/step - loss: 6.9290e-04 - mse

In [23]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_24-6\assets


984/984 [==============================] - 1s 1ms/step
mean_squared_error
train set: 0.0005670177831102061
test set: 0.0006549832945472625


## input length : output length = 32:8

In [24]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [25]:
# create model instance
model_name = 'BiLSTM_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_32-8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 8)                 488       
                                                                 
Total params: 10,808
Trainable params: 10,808
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 14s 11ms/step - loss: 0.0019 - mse: 0.0019 - acc: 0.5011 - val_loss: 0.0015 - val_mse: 0.0015 - val_acc: 0.4676
Epoch 2/30
1135/1135 [==============================] - 11s 10ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5412 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5240
Epoch 3/30
1135/1135 [==============================] - 11s 10ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5546 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5472
Epoch 4/30
1135/1135 [==============================] - 11s 10ms/step - loss: 0.0010 - mse: 0.0010 - acc: 0.5600 - val_loss: 0.0010 - val_mse: 0.0010 - val_acc: 0.5523
Epoch 5/30
1135/1135 [==============================] - 12s 10ms/step - loss: 9.8514e-04 - mse: 9.8514e-04 - acc: 0.5641 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5306
Epoch 6/30
1135/1135 [==============================] - 12s 10ms/step - loss: 9.6318e-04 - mse: 9.6318e-04 - acc: 0.5673 - val_loss: 0.0011 - val_mse: 0

In [27]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_32-8\assets


984/984 [==============================] - 2s 2ms/step
mean_squared_error
train set: 0.0008149588978308926
test set: 0.0009227961871954862


## input length : output length = 40:10

In [28]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [29]:
# create model instance
model_name = 'BiLSTM_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_40-10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 40, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 10)                610       
                                                                 
Total params: 10,930
Trainable params: 10,930
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)


Epoch 1/30
1135/1135 [==============================] - 17s 13ms/step - loss: 0.0021 - mse: 0.0021 - acc: 0.4765 - val_loss: 0.0015 - val_mse: 0.0015 - val_acc: 0.4831
Epoch 2/30
1135/1135 [==============================] - 14s 12ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.5140 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.4997
Epoch 3/30
1135/1135 [==============================] - 14s 12ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5183 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.5265
Epoch 4/30
1135/1135 [==============================] - 14s 12ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5238 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.5196
Epoch 5/30
1135/1135 [==============================] - 14s 12ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5250 - val_loss: 0.0013 - val_mse: 0.0013 - val_acc: 0.4927
Epoch 6/30
1135/1135 [==============================] - 14s 12ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5279 - val_loss: 0.0013 - val_mse: 0.0013 - val_acc:

In [31]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models/BiLSTM_40-10\assets


984/984 [==============================] - 2s 2ms/step
mean_squared_error
train set: 0.0010243174063823885
test set: 0.0011515796027563208
